In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Convolution1D,MaxPooling1D, Flatten

from keras import backend as K
from sklearn.cross_validation import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.utils import np_utils

import h5py

X = np.load("mawi_features.pkl")
Y = np.load("mawi_labels.pkl")
C = np.load("mawi_labels.pkl")
T = np.load("mawi_features.pkl")

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train = np.array(Y)
y_test = np.array(C)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))

from keras.utils import multi_gpu_model

# Replicates `model` on 5 GPUs.
# This assumes that your machine has 5 available GPUs.
cnn = multi_gpu_model(model, gpus=5)

Using TensorFlow backend.
C:\Anaconda\envs\GPU\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


['/gpu:0']

In [2]:
cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(15, 1)))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(Flatten())
cnn.add(Dense(128, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(2, activation="sigmoid"))
print(cnn.summary())
# define optimizer and objective, compile cnn

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 15, 64)            256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 7, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 448)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               57472     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 57,986
Trainable params: 57,986
Non-trainable params: 0
_________________________________________________________________
None


C:\Anaconda\envs\GPU\lib\site-packages\ipykernel\__main__.py:2: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(15, 1), padding="same")`
  from ipykernel import kernelapp as app
C:\Anaconda\envs\GPU\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  app.launch_new_instance()


In [3]:
cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
cnn.fit(X_train, y_train, epochs=500,validation_data=(X_test, y_test))
# serialize model to JSON
model_json = model.to_json()
with open("cnn1D_MawiLAB.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("cnn1D_MawiLAB.h5")
print("Saved model to disk")

Train on 383615 samples, validate on 383615 samples
Epoch 1/500
383615/383615 [==============================] - 97s 253us/step - loss: 0.4575 - acc: 0.7265 - val_loss: 0.4378 - val_acc: 0.7399
Epoch 2/500
383615/383615 [==============================] - 96s 251us/step - loss: 0.4358 - acc: 0.7451 - val_loss: 0.4190 - val_acc: 0.7489
Epoch 3/500
383615/383615 [==============================] - 90s 234us/step - loss: 0.4225 - acc: 0.7526 - val_loss: 0.4107 - val_acc: 0.7578
Epoch 4/500
383615/383615 [==============================] - 84s 218us/step - loss: 0.4184 - acc: 0.7524 - val_loss: 0.4082 - val_acc: 0.7572
Epoch 5/500
383615/383615 [==============================] - 84s 219us/step - loss: 0.4163 - acc: 0.7520 - val_loss: 0.4065 - val_acc: 0.7488
Epoch 6/500
383615/383615 [==============================] - 85s 221us/step - loss: 0.4150 - acc: 0.7530 - val_loss: 0.4068 - val_acc: 0.7578
Epoch 7/500
383615/383615 [==============================] - 84s 219us/step - loss: 0.4135 - acc

KeyboardInterrupt: 